In [54]:
import requests
from requests.auth import HTTPBasicAuth 
BASE_URL = "http://localhost:2480"
AUTH = HTTPBasicAuth("root", "playwithdata")

Create a database "testdb"

In [55]:
def create_database(name):
    url = f"{BASE_URL}/api/v1/server"
    payload = { "command": f"create database {name}" }
    response = requests.post(url, json=payload, auth=AUTH)
    print(f"[CREATE DB] Status: {response.status_code}\nResponse: {response.text}\n")

create_database("testdb")

[CREATE DB] Status: 400
Response: {"error":"Cannot execute command","detail":"Database \u0027testdb\u0027 already exists","exception":"java.lang.IllegalArgumentException"}



Run queries 

In [56]:
def run_cypher(query:str):
    url = f"{BASE_URL}/api/v1/command/testdb"
    payload = {"language":"Cypher","command":query}
    response = requests.post(url,json = payload,auth = AUTH)
    print(f"[RUN CYPHER] Status:{response.status_code}\n Response :{response.text}\n")
    return response.json()

In [ ]:
Create vertices

In [8]:
create_nodes_query = """
CREATE (:Person {name: 'Alice', age: 25})
CREATE (:Person {name: 'Bob', age: 30})
CREATE (:Person {name: 'Charlie', age: 35})
"""

result = run_cypher(create_nodes_query)
print("Nodes Created:", result)

[RUN CYPHER] Status:200
 Response :{"result":[]}

Nodes Created: {'result': []}


Create Realtions 

In [11]:
# Create Alice -> Bob friendship
run_cypher("""
MATCH (a:Person {name: 'Alice'}), (b:Person {name: 'Bob'})
CREATE (a)-[:FRIEND]->(b)


MATCH (b:Person {name: 'Bob'}), (c:Person {name: 'Charlie'})
CREATE (b)-[:FRIEND]->(c)
""")


[RUN CYPHER] Status:500
 Response :{"error":"Error on transaction commit","detail":"Error on executing Cypher query","exception":"com.arcadedb.exception.CommandParsingException"}



{'error': 'Error on transaction commit',
 'detail': 'Error on executing Cypher query',
 'exception': 'com.arcadedb.exception.CommandParsingException'}

In [13]:
create_relationships_query =[ """
MATCH (a:Person {name: 'Alice'}), (b:Person {name: 'Bob'})
CREATE (a)-[:FRIEND]->(b)
""","""
MATCH (b:Person {name: 'Bob'}), (c:Person {name: 'Charlie'})
CREATE (b)-[:FRIEND]->(c)
"""]
for q in create_relationships_query:
    result = run_cypher(q)
    print("Relationships Created:", result)

[RUN CYPHER] Status:200
 Response :{"result":[]}

Relationships Created: {'result': []}
[RUN CYPHER] Status:200
 Response :{"result":[]}

Relationships Created: {'result': []}


In [21]:
query_friends = """
MATCH (a:Person)-[:FRIEND]->(b:Person)
RETURN a.name AS person, b.name AS friend

"""

result = run_cypher(query_friends)
for row in result.get("result", []):
    print(f"{row['person']} is friends with {row['friend']}")


[RUN CYPHER] Status:200
 Response :{"result":[{"person":"Alice","friend":"Bob"},{"person":"Alice","friend":"Bob"},{"person":"uday","friend":"jay"},{"person":"uday","friend":"kay"},{"person":"kay","friend":"Diana"},{"person":"jay","friend":"Eve"},{"person":"Eve","friend":"uday"}]}

Alice is friends with Bob
Alice is friends with Bob
uday is friends with jay
uday is friends with kay
kay is friends with Diana
jay is friends with Eve
Eve is friends with uday


In [15]:
result

{'result': [{'person': 'Alice', 'friend': 'Bob'},
  {'person': 'Alice', 'friend': 'Bob'},
  {'person': 'Bob', 'friend': 'Charlie'},
  {'person': 'Bob', 'friend': 'Charlie'}]}

Delete edges

In [16]:
run_cypher("""
MATCH (p:Person {name: 'Charlie'})-[r]-()
DELETE r
""")

[RUN CYPHER] Status:200
 Response :{"result":[]}



{'result': []}

Delete vertex

In [17]:
run_cypher("""
MATCH (p:Person {name: 'Charlie'})
DELETE p
""")

[RUN CYPHER] Status:200
 Response :{"result":[]}



{'result': []}

In [19]:
create_node = """
CREATE (:Person {
  name: 'vamsi',
  age: 23,
  email: 'vamsi@example.com',
  location: 'Hyderabad',
  skills: ['Python', 'AI', 'ML'],
  isEngineer: true
})
"""

print(run_cypher(create_node))

[RUN CYPHER] Status:200
 Response :{"result":[]}

{'result': []}


In [20]:
create_more_nodes = """
CREATE (a:Person {name: 'uday', age: 24}),
       (b:Person {name: 'kay', age: 28}),
       (c:Person {name: 'jay', age: 30}),
       (d:Person {name: 'Diana', age: 26}),
       (e:Person {name: 'Eve', age: 27})

CREATE (a)-[:FRIEND]->(b),
       (a)-[:FRIEND]->(c),
       (b)-[:FRIEND]->(d),
       (c)-[:FRIEND]->(e),
       (e)-[:FRIEND]->(a)
"""
run_cypher(create_more_nodes)

[RUN CYPHER] Status:200
 Response :{"result":[]}



{'result': []}

Query for creating vertex

In [23]:
def format_properties(properties: dict) -> str:
    return '{' + ', '.join(f"{k}: '{v}'" for k, v in properties.items()) + '}'


def create_vertex(label: str, properties: dict):
    props = format_properties(properties)
    query = f"CREATE (:{label} {props})"
    print(f"[Create Vertex] {label}: {properties}")
    return run_cypher(query)

Query for creating relationship

In [24]:
def create_relationship(from_label: str, from_props: dict,
                        to_label: str, to_props: dict,
                        rel_type: str, rel_props: dict = None):
    from_match = format_properties(from_props)
    to_match = format_properties(to_props)
    rel_props_str = format_properties(rel_props) if rel_props else ''
    query = f"""
    MATCH (a:{from_label} {from_match}), (b:{to_label} {to_match})
    CREATE (a)-[:{rel_type} {rel_props_str}]->(b)
    """
    print(f"[Create Relationship] {from_props} -[{rel_type}]-> {to_props}")
    return run_cypher(query)

In [35]:
# Company data
companies = [
    {"name": "OpenAI", "location": "San Francisco"},
    {"name": "Google", "location": "Mountain View"},
    {"name": "Microsoft", "location": "Redmond"}
]

# Relations: person name → company name
person_company_map = {
    "Alice": "OpenAI",
    "Bob": "Google",
    "Charlie": "Microsoft",
    "Diana": "OpenAI",
    "Eve": "Google",
    "vamsi": "Microsoft"
}

In [36]:
# 1. Create company vertices
for company in companies:
    create_vertex("Company", company)

# 2. Create WORKS_IN relationships
for person, company in person_company_map.items():
    create_relationship(
        from_label="Person", from_props={"name": person},
        to_label="Company", to_props={"name": company},
        rel_type="WORKS_IN"
    )

[Create Vertex] Company: {'name': 'OpenAI', 'location': 'San Francisco'}
[RUN CYPHER] Status:200
 Response :{"result":[]}

[Create Vertex] Company: {'name': 'Google', 'location': 'Mountain View'}
[RUN CYPHER] Status:200
 Response :{"result":[]}

[Create Vertex] Company: {'name': 'Microsoft', 'location': 'Redmond'}
[RUN CYPHER] Status:200
 Response :{"result":[]}

[Create Relationship] {'name': 'Alice'} -[WORKS_IN]-> {'name': 'OpenAI'}
[RUN CYPHER] Status:200
 Response :{"result":[]}

[Create Relationship] {'name': 'Bob'} -[WORKS_IN]-> {'name': 'Google'}
[RUN CYPHER] Status:200
 Response :{"result":[]}

[Create Relationship] {'name': 'Charlie'} -[WORKS_IN]-> {'name': 'Microsoft'}
[RUN CYPHER] Status:200
 Response :{"result":[]}

[Create Relationship] {'name': 'Diana'} -[WORKS_IN]-> {'name': 'OpenAI'}
[RUN CYPHER] Status:200
 Response :{"result":[]}

[Create Relationship] {'name': 'Eve'} -[WORKS_IN]-> {'name': 'Google'}
[RUN CYPHER] Status:200
 Response :{"result":[]}

[Create Relationsh

Modifying creating vertex and edges, 
only create vertices that are not present  #using merge 
# creating vertices 

In [ ]:
def format_properties(properties: dict) -> str:
    return '{' + ', '.join(f"{k}: '{v}'" for k, v in properties.items()) + '}'

In [46]:
def create_new_vertex(label: str, properties: dict = None):
    if properties:
        props = format_properties(properties)
        query = f"MERGE (:{label} {props})"
    else:
        query = f"MERGE (:{label})"

    print(f"[MERGE Vertex] {label}: {properties}")
    return run_cypher(query)


def create_new_relationship(from_label: str, from_props: dict,
                        to_label: str, to_props: dict,
                        rel_type: str, rel_props: dict = None):
    from_match = format_properties(from_props)
    to_match = format_properties(to_props)
    rel_props_str = format_properties(rel_props) if rel_props else ''

    query = f"""
    MATCH (a:{from_label} {from_match}), (b:{to_label} {to_match})
    MERGE (a)-[r:{rel_type} {rel_props_str}]->(b)
    """
    print(f"[MERGE Relationship] {from_props} -[{rel_type}]-> {to_props}")
    return run_cypher(query)

# This should be created by LLM so we have to provide the labels ahead 

In [69]:
create_new_vertex("college",{"name":"IIT Dhanbad","location":"Dhanbad","tier":1}) 

[MERGE Vertex] college: {'name': 'IIT Dhanbad', 'location': 'Dhanbad', 'tier': 1}
[RUN CYPHER] Status:200
 Response :{"result":[]}



{'result': []}

In [71]:
create_new_relationship("Person",{"name":"vamsi"},"college",{"name":"IIT Dhanbad"},"STUDIED_IN")

[MERGE Relationship] {'name': 'vamsi'} -[STUDIED_IN]-> {'name': 'IIT Dhanbad'}
[RUN CYPHER] Status:200
 Response :{"result":[]}



{'result': []}

In [48]:
def delete_vertex(label: str, properties: dict = None, detach: bool = True):
    match_clause = f"(n:{label}"

    if properties:
        props = format_properties(properties)
        match_clause += f" {props}"
    match_clause += ")"

    delete_clause = "DETACH DELETE n" if detach else "DELETE n"
    query = f"MATCH {match_clause} {delete_clause}"

    print(f"[DELETE Vertex] Query: {query}")
    return run_cypher(query)


In [49]:
delete_vertex("location")

[DELETE Vertex] Query: MATCH (n:vamsi) DETACH DELETE n
[RUN CYPHER] Status:200
 Response :{"result":[]}



{'result': []}

In [50]:
def delete_edge(from_label, from_props, rel_type, to_label, to_props):    from_clause = f"(a:{from_label} {format_properties(from_props)})"
    to_clause = f"(b:{to_label} {format_properties(to_props)})"
    query = f"""
        MATCH {from_clause}-[r:{rel_type}]->{to_clause}
        DELETE r
    """
    print(f"[DELETE Edge] Query: {query}")
    return run_cypher(query)

In [51]:
# Delete a WORKS_AT relationship from 'Alice' to 'OpenAI'
delete_edge(
    from_label="Person",
    from_props={"name": "Alice"},
    rel_type="FRIEND",
    to_label="Person",
    to_props={"name": "Bob"}
)

[DELETE Edge] Query: 
        MATCH (a:Person {name: 'Alice'})-[r:FRIEND]->(b:Person {name: 'Bob'})
        DELETE r
    
[RUN CYPHER] Status:200
 Response :{"result":[]}



{'result': []}

In [67]:
run_cypher("""MATCH (u:Person {name: 'Bob'})-[:WORKS_IN]->(c1:Company)
OPTIONAL MATCH (u)-[:FRIEND]->(f:Person)-[:WORKS_IN]->(c2:Company)
RETURN u.name AS person, c1.name AS Bob_company, f.name AS friend, c2.name AS friend_company
""")

[RUN CYPHER] Status:200
 Response :{"result":[{"person":"Bob","Bob_company":"Google","friend":null,"friend_company":null}]}



{'result': [{'person': 'Bob',
   'Bob_company': 'Google',
   'friend': None,
   'friend_company': None}]}

In [61]:
run_cypher("MATCH (n) WHERE n.name = 'uday' RETURN n")

[RUN CYPHER] Status:200
 Response :{"result":[{"@rid":"#1:3","@type":"Person","name":"uday","age":24}]}



{'result': [{'@rid': '#1:3', '@type': 'Person', 'name': 'uday', 'age': 24}]}

In [81]:
run_cypher("""MATCH (n)
RETURN DISTINCT labels(n) AS labels
"""
)

[RUN CYPHER] Status:200
 Response :{"result":[{"labels":["Person"]},{"labels":["Company"]},{"labels":["college"]}]}



{'result': [{'labels': ['Person']},
  {'labels': ['Company']},
  {'labels': ['college']}]}

In [78]:
run_cypher("""
MATCH ()-[r]->()
RETURN DISTINCT type(r) AS relationship_type""")

[RUN CYPHER] Status:200
 Response :{"result":[{"relationship_type":"FRIEND"},{"relationship_type":"WORKS_IN"},{"relationship_type":"STUDIED_IN"}]}



{'result': [{'relationship_type': 'FRIEND'},
  {'relationship_type': 'WORKS_IN'},
  {'relationship_type': 'STUDIED_IN'}]}

In [79]:
run_cypher("""
MATCH (n:Person)
RETURN DISTINCT keys(n) AS properties

""")

[RUN CYPHER] Status:200
 Response :{"result":[{"properties":["name","age"]},{"properties":["name","age","email","location","skills","isEngineer"]}]}



{'result': [{'properties': ['name', 'age']},
  {'properties': ['name',
    'age',
    'email',
    'location',
    'skills',
    'isEngineer']}]}

In [83]:
# checking for the existing vertices with difflib

In [82]:
import difflib

# Maintain a set of existing labels (ideally fetched from DB once)
existing_labels = {"Person", "Company", "City"}

def get_closest_label(label, existing_labels, threshold=0.8):
    """Returns closest existing label if similarity is high enough."""
    matches = difflib.get_close_matches(label, existing_labels, n=1, cutoff=threshold)
    return matches[0] if matches else None

def format_properties(properties: dict) -> str:
    if not properties:
        return ""
    return '{' + ', '.join(f"{k}: '{v}'" for k, v in properties.items()) + '}'

def create_vertex_smart(label: str, properties: dict = None):
    # Try to find a close existing label
    matched_label = get_closest_label(label, existing_labels)
    if matched_label:
        used_label = matched_label
        print(f"Using existing/close label: {matched_label}")
    else:
        used_label = label
        existing_labels.add(label)  # Track new label
        print(f"New label created: {label}")

    props = format_properties(properties)
    query = f"MERGE (:{used_label} {props})"
    print(f"[Cypher Query] {query}")
    return query  # You can pass it to run_cypher(query)


In [86]:
import difflib

class GraphTypeCache:
    def __init__(self):
        self.vertex_types = set()
        self.edge_types = set()
        self.similarity_threshold = 0.8  # can be adjusted

    def load_from_db(self):
        label_query = "MATCH (n) RETURN DISTINCT labels(n) AS labels"
        label_result = run_cypher(label_query)
        self.vertex_types = {
            label for row in label_result['result'] for label in row['labels']
        }

        rel_query = "MATCH ()-[r]->() RETURN DISTINCT type(r) AS relationship_type"
        rel_result = run_cypher(rel_query)
        self.edge_types = {
            row['relationship_type'] for row in rel_result['result']
        }

    def find_closest_vertex_type(self, label: str) -> str:
        match = difflib.get_close_matches(label, self.vertex_types, n=1, cutoff=self.similarity_threshold)
        return match[0] if match else label

    def find_closest_edge_type(self, rel: str) -> str:
        match = difflib.get_close_matches(rel, self.edge_types, n=1, cutoff=self.similarity_threshold)
        return match[0] if match else rel

    def add_vertex_type(self, label: str):
        self.vertex_types.add(label)

    def add_edge_type(self, rel: str):
        self.edge_types.add(rel)


In [87]:
def create_vertex_if_not_exists(label: str, properties: dict = None):
    label = type_cache.find_closest_vertex_type(label)

    if label not in type_cache.vertex_types:
        type_cache.add_vertex_type(label)

    props = format_properties(properties) if properties else "{}"
    query = f"MERGE (:{label} {props})"
    print(f"[MERGE Vertex] {label}: {props}")
    run_cypher(query)


In [88]:
def create_edge_if_not_exists(from_label: str, from_props: dict,
                              to_label: str, to_props: dict,
                              relation: str):
    from_label = type_cache.find_closest_vertex_type(from_label)
    to_label = type_cache.find_closest_vertex_type(to_label)
    relation = type_cache.find_closest_edge_type(relation)

    # Update cache if not matched before
    if from_label not in type_cache.vertex_types:
        type_cache.add_vertex_type(from_label)
    if to_label not in type_cache.vertex_types:
        type_cache.add_vertex_type(to_label)
    if relation not in type_cache.edge_types:
        type_cache.add_edge_type(relation)

    from_props_str = format_properties(from_props)
    to_props_str = format_properties(to_props)

    query = f"""
    MATCH (a:{from_label} {from_props_str}), (b:{to_label} {to_props_str})
    MERGE (a)-[r:{relation}]->(b)
    """
    print(f"[MERGE Edge] ({from_label})-[:{relation}]->({to_label})")
    run_cypher(query)


In [89]:
type_cache = GraphTypeCache()
type_cache.load_from_db()


[RUN CYPHER] Status:200
 Response :{"result":[{"labels":["Person"]},{"labels":["Company"]},{"labels":["college"]}]}

[RUN CYPHER] Status:200
 Response :{"result":[{"relationship_type":"FRIEND"},{"relationship_type":"WORKS_IN"},{"relationship_type":"STUDIED_IN"}]}



In [91]:
create_vertex_if_not_exists("eople",{"name":"rattu"})

[MERGE Vertex] People: {name: 'rattu'}
[RUN CYPHER] Status:200
 Response :{"result":[]}



In [ ]:
def create_new_vertex(label: str, properties: dict = None):
    global known_vertex_labels

    matched_label = get_semantic_match_llm(label, known_vertex_labels)

    if matched_label == "NEW":
        logger.info(f"Adding new label: {label}")
        known_vertex_labels.add(label)
        save_cache(known_vertex_labels, VERTEX_CACHE_FILE)
        final_label = label
    else:
        final_label = matched_label

    props = format_properties(properties)
    query = f"MERGE (:{final_label} {props})"

    logger.info(f"Creating vertex: {final_label} {properties}")
    run_cypher(query)


In [92]:
def create_relationship(label1: str, props1: dict, relation: str, label2: str, props2: dict):
    global known_vertex_labels, known_relationship_types

    label1_match = get_semantic_match_llm(label1, known_vertex_labels)
    if label1_match == "NEW":
        known_vertex_labels.add(label1)
        save_cache(known_vertex_labels, VERTEX_CACHE_FILE)
        label1_match = label1
        logger.info(f"New vertex label added: {label1}")

    label2_match = get_semantic_match_llm(label2, known_vertex_labels)
    if label2_match == "NEW":
        known_vertex_labels.add(label2)
        save_cache(known_vertex_labels, VERTEX_CACHE_FILE)
        label2_match = label2
        logger.info(f"New vertex label added: {label2}")

    rel_match = get_semantic_match_llm(relation, known_relationship_types)
    if rel_match == "NEW":
        known_relationship_types.add(relation)
        save_cache(known_relationship_types, EDGE_CACHE_FILE)
        rel_match = relation
        logger.info(f"New relationship type added: {relation}")

    props1_str = format_properties(props1)
    props2_str = format_properties(props2)

    query = (
        f"MATCH (a:{label1_match} {props1_str}), (b:{label2_match} {props2_str}) "
        f"MERGE (a)-[:{rel_match}]->(b)"
    )

    logger.info(f"Creating relationship: ({label1_match})-[:{rel_match}]->({label2_match})")
    run_cypher(query)


In [93]:
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

def generate_and_run_query_llm(query_structure: list[dict]):
    """
    Uses LLM to semantically map query structure to existing graph types, 
    generates Cypher, and executes it.

    :param query_structure: List of nodes and edges like:
        [
            {"label": "Person", "properties": {"name": "Uday"}},
            {"relation": "WORKS_IN"},
            {"label": "Company"}
        ]
    """
    if not query_structure or len(query_structure) < 3:
        logger.error("Invalid query structure: must include at least 2 vertices and a relation.")
        return

    cypher_parts = []
    var_counter = 0
    previous_var = None

    for item in query_structure:
        if "label" in item:
            label = item["label"]
            props = item.get("properties", {})

            # Semantic match label
            matched_label = get_semantic_match_llm(label, known_vertex_labels)
            if matched_label == "NEW":
                logger.warning(f"No match found for vertex label '{label}', using raw label.")
                matched_label = label

            # Format properties
            if props:
                props_str = "{" + ", ".join(f"{k}: '{v}'" for k, v in props.items()) + "}"
            else:
                props_str = ""

            var_name = f"v{var_counter}"
            cypher_parts.append(f"({var_name}:{matched_label} {props_str})")
            previous_var = var_name
            var_counter += 1

        elif "relation" in item:
            relation = item["relation"]

            matched_rel = get_semantic_match_llm(relation, known_relationship_types)
            if matched_rel == "NEW":
                logger.warning(f"No match found for relation '{relation}', using raw name.")
                matched_rel = relation

            next_var = f"v{var_counter}"
            cypher_parts.append(f"-[:{matched_rel}]->")
            previous_var = next_var  # for chaining
        else:
            logger.error(f"Unknown query component: {item}")
            return

    full_match = "MATCH " + "".join(cypher_parts)
    return_stmt = "RETURN " + ", ".join(f"v{i}" for i in range(var_counter))

    final_query = f"{full_match} {return_stmt}"
    logger.info(f"Generated Cypher Query:\n{final_query}")

    return run_cypher(final_query)


In [94]:
query_structure = [
    {"label": "Person", "properties": {"name": "Uday"}},
    {"relation": "WORKS_IN"},
    {"label": "Company"}
]

generate_and_run_query_llm(query_structure)


NameError: name 'get_semantic_match_llm' is not defined